In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go
import datetime
import instrument

In [2]:
symbol = "PF_ETHUSD"
interval = "1h"
ma_list = [16, 64]
i_symbol = instrument.Instrument.get_instrument_by_name(symbol)

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(symbol, interval))
mod_cols = [x for x in df.columns if x != 'time']
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [4]:
df_ma = df.copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.close.rolling(window=ma).mean()

In [5]:
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)
df_ma.head()

,time,volume,open,high,low,close,MA_16,MA_64
0,2023-06-05 10:00:00,174.185,1870.7,1872.3,1865.1,1865.1,1882.36875,1896.387500
1,2023-06-05 11:00:00,1864.925,1865.1,1874.0,1864.5,1872.8,1880.29375,1895.826563
2,2023-06-05 12:00:00,288.182,1872.8,1872.8,1863.4,1867.4,1877.92500,1895.234375
3,2023-06-05 13:00:00,773.835,1867.4,1872.4,1861.6,1870.0,1875.88125,1894.675000
4,2023-06-05 14:00:00,58.996,1870.0,1874.1,1866.3,1870.0,1873.84375,1894.092187


In [6]:
def ma_cross(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    else:
        return 0

In [7]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['MA_CROSS'] = df_ma.apply(ma_cross, axis=1)
df_crosses = df_ma[df_ma.MA_CROSS != 0].copy()

In [8]:
df_crosses["DELTA"] = df_crosses.close.diff().shift(-1)
df_crosses["GAIN"] = df_crosses["DELTA"] * df_crosses["MA_CROSS"]

In [9]:
df_crosses["DURATION"] = df_crosses.time.diff().shift(-1)

In [15]:
df_crosses["DURATION"] = [x.total_seconds() / 3600 for x in df_crosses.DURATION]

In [16]:
df_crosses.head()

,time,volume,open,high,low,close,MA_16,MA_64,DIFF,DIFF_PREV,MA_CROSS,DELTA,GAIN,DURATION
39,2023-06-07 01:00:00,186.629,1887.2,1887.2,1878.6,1880.0,1859.15000,1858.546875,0.603125,-3.590625,1,-43.3,-43.3,26.0
65,2023-06-08 03:00:00,161.159,1838.8,1838.8,1830.0,1836.7,1843.67500,1844.407812,-0.732812,0.834375,-1,-95.3,95.3,154.0
219,2023-06-14 13:00:00,161.433,1746.6,1747.0,1738.6,1741.4,1743.37500,1743.300000,0.075000,-0.395312,1,-12.1,-12.1,6.0
225,2023-06-14 19:00:00,809.786,1734.9,1740.8,1727.5,1729.3,1741.58125,1742.545312,-0.964062,0.028125,-1,-2.2,2.2,49.0
274,2023-06-16 20:00:00,125.377,1722.4,1728.1,1719.0,1727.1,1682.47500,1680.118750,2.356250,-1.598438,1,-4.1,-4.1,60.0


In [10]:
df_crosses["GAIN"].sum()

-30.099999999999

In [17]:
df_all = pd.read_pickle("all_crosses.pkl")

In [19]:
df_all

,time,close,DIFF,DIFF_PREV,MA_CROSS,DELTA,GAIN,SYMBOL,MASHORT,MALONG,DURATION
11,2023-06-03 06:00:00,0.37770,0.000267,-0.000273,1,-0.00190,-0.00190,PF_ADAUSD,4,8,3.0
14,2023-06-03 09:00:00,0.37580,-0.000021,0.000260,-1,0.00171,-0.00171,PF_ADAUSD,4,8,6.0
20,2023-06-03 15:00:00,0.37751,0.000117,-0.000075,1,-0.00316,-0.00316,PF_ADAUSD,4,8,4.0
24,2023-06-03 19:00:00,0.37435,-0.000438,0.000128,-1,0.00171,-0.00171,PF_ADAUSD,4,8,5.0
29,2023-06-04 00:00:00,0.37606,0.000106,-0.000417,1,0.00484,0.00484,PF_ADAUSD,4,8,13.0
...,...,...,...,...,...,...,...,...,...,...,...
2540,2023-09-16 15:00:00,0.49930,0.000181,-0.000095,1,0.00605,0.00605,PF_XRPUSD,64,256,224.0
2764,2023-09-25 23:00:00,0.50535,-0.000069,0.000078,-1,0.01526,-0.01526,PF_XRPUSD,64,256,95.0
2859,2023-09-29 22:00:00,0.52061,0.000040,-0.000258,1,-0.02181,-0.02181,PF_XRPUSD,64,256,228.0
3087,2023-10-09 10:00:00,0.49880,-0.000118,0.000252,-1,0.02104,-0.02104,PF_XRPUSD,64,256,253.0


In [20]:
df_all.GAIN.sum()

55614.07343900001

In [11]:
df_plot = df_ma.iloc[:200].copy()

In [12]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time,
    open = df_plot.open,
    high = df_plot.high,
    low = df_plot.low,
    close = df_plot.close,
    line=dict(width=1),
    opacity = 1,
    increasing_fillcolor = '#24A06B',
    decreasing_fillcolor = '#CC2E3C',
    increasing_line_color = '#2EC886',
    decreasing_line_color = '#FF3A4C'
))
for ma in ma_list:
    col = f'MA_{ma}'
    fig.add_trace(go.Scatter(
        x = df_plot.time,
        y = df_plot[col],
        line = dict(width = 2),
        line_shape = 'spline',
        name = col
))
fig.update_layout(width = 1000, height = 400,
                  margin = dict(l=10, r=10, b=10, t=10),
                  font = dict(size=10, color='#e1e1e1'),
                  paper_bgcolor = '#1e1e1e',
                  plot_bgcolor = '#1e1e1e'
                  )
fig.update_xaxes(
    gridcolor = '#1f292f',
    showgrid=True, fixedrange=True, rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)
fig.show()